### Imports

In [5]:
import pandas as pd
import numpy as np
import csv
import datetime
import os

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [183]:
import shutil
import collections
from collections import defaultdict

### Path Set up
There should be paths that direct to the completed csv annotations, the directory in which the behavior clips should be saved (with corresponding folders related to each behavior), along with the path to the hour long videos. 

Additionally, in the case of testing, there should be a path to the parent test directory.

In [1]:
#directory containing all csv files for annotations
parent_dir = '/Users/graciezhang/Documents/Hengen-Lab/annotations/completed/csv'

In [6]:
#current working directory
cwd = os.getcwd()

In [7]:
#where all behavior clips should be stored
clip_dir = os.path.join(cwd, 'behavior_clips')

In [8]:
#where original videos can be accessed
vid_dir = os.path.join(cwd, 'videos')

In [9]:
#video split directory
test_dir = os.path.join(cwd, 'behavior_labeling')

### Key
Encodes all categorical behaviors.

In [10]:
#encodes all categorical behaviors to numerical
def cat_num(df):
    #encode
    df.Behavior[df.Behavior=='Nesting'] = 1 # Location
    df.Behavior[df.Behavior=='Playing w/ Ball'] = 2 # Location
    df.Behavior[df.Behavior=='Playing w/ Box'] = 3 # Location
    df.Behavior[df.Behavior=='Riding the Ball'] = 4 # Location
    df.Behavior[df.Behavior=='Riding the Box'] = 5 # Location
    df.Behavior[df.Behavior=='Drinking'] = 6 # Location
    df.Behavior[df.Behavior=='Chewing'] = 7 # Location
    df.Behavior[df.Behavior=='Porthole Interaction'] = 8 # Location
    df.Behavior[df.Behavior=='Scaling Porthole'] = 9 #Location
    df.Behavior[df.Behavior=='Scaling Door'] = 10 #Location
    
    df.Behavior[df.Behavior=='Grooming'] = 11
    
    df.Behavior[df.Behavior=='Hollowing/Digging'] = 12
    df.Behavior[df.Behavior=='Digging'] = 12
    
    df.Behavior[df.Behavior=='Locomotion'] = 13
    df.Behavior[df.Behavior=='Locomotive Movement'] = 13
    
    df.Behavior[df.Behavior=='Minor Postural Movement'] = 14
    df.Behavior[df.Behavior=='Rearing'] = 15
    df.Behavior[df.Behavior=='Stretching'] = 16
    df.Behavior[df.Behavior=='Twitching'] = 17
    df.Behavior[df.Behavior=='Slipping'] = 18
    
    #drop all nonbehaviors
    df = df[df.Behavior.apply(lambda x: str(x).isnumeric())]
    
    return df

### Load in behavior dataframes
Given the file path of the csv, several preprocessing steps occur. First, only the annotations are read in from the file. Additionally, the video name is extracted. Unnecessary columns are dropped and behaviors are encoded. Nonbehaviors are added in.

In [11]:
def get_video_path(file_name):
    with open(file_name, newline='') as f:
      reader = csv.reader(f)
      for row in reader:
            if row[0] == "Player #1":
                return row[1].split("/")[-1]
    return None

In [12]:
def get_skip_rows(file_name):
    with open(file_name, newline='') as f:
      reader = csv.reader(f)
      for i, row in enumerate(reader):
            if row[0] == "Time":
                return i
    return None

In [13]:
def clean_ants(file_name):
    skip = get_skip_rows(file_name)
    video_name = get_video_path(file_name)
    
    df = pd.read_csv(file_name, skiprows = skip)
    
    #drop unnecessary columns
    drop_filter = df.filter(["Media file path", "Subject", "Behavioral category", "Comment"])
    
    df.drop(drop_filter, axis = 1, inplace = True)
    
    #encode all categorical behaviors
    df = cat_num(df)
    
    return df, video_name

### Produce dataframes

In [14]:
VIDEO_NAMES = []

In [15]:
dataframes = {}
for file in os.listdir(parent_dir):
    if file.endswith(".csv"):
        df, video_name = clean_ants(os.path.join(parent_dir, file))
        dataframes[video_name] = df
        
        #only need to include in one run
        #VIDEO_NAMES.append(video_name)

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Nesting'] = 1 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Playing w/ Ball'] = 2 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Drinking'] = 6 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Chewing'] = 7 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Grooming'] = 11
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Hollowing/Digging'] = 12
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Riding the Box'] = 5 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Drinking'] = 6 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Stretching'] = 16
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Twitching'] = 17
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Playing w/ Box'] = 3 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Riding the Ball'] = 4 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Digging'] = 12
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Locomotion'] = 13
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Nesting'] = 1 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Playing w/ Ball'] = 2 # Location
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Grooming'] = 11
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Behavior[df.Behavior=='Hollowing/Digging'] = 12
/var/folders/99/r5sn_bd95q3fczxk1xbsy_kh0000gn/T/ipykernel_1182/3889278364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [58]:
#key: video name, value: dataframe of mouse annotations
len(dataframes)

12

### Check dataframes + variables

In [48]:
# #clears all behavior_clips directories
# for directory in os.listdir(clip_dir):
#     parent = os.path.join(clip_dir, directory)
#     if os.path.isdir(parent):
#         for file in os.listdir(parent):
#             if file.startswith("."):
#                 continue

#             else: 
#                 os.remove(os.path.join(parent, file))

In [50]:
# #loads in all behaviors + corresponding folders - may vary depending on ur file system
# BEHAVIOR_NAMES = [name for name in os.listdir(clip_dir) if os.path.isdir(os.path.join(clip_dir, name))]
# BEHAVIORS = {}
# for behavior in BEHAVIOR_NAMES:
#     num = behavior.split("_")[0]
#     name = behavior
    
#     BEHAVIORS[num] = name

In [52]:
BEHAVIORS

{'11': '11_grooming',
 '2': '2_playing_w_ball',
 '7': '7_chewing',
 '5': '5_riding_box',
 '9': '9_scaling_porthole',
 '8': '8_porthole_interaction',
 '13': '13_locomotion',
 '18': '18_slipping',
 '12': '12_digging',
 '14': '14_minor_postural_movement',
 '4': '4_riding_ball',
 '3': '3_playing_w_box',
 '15': '15_rearing',
 '16': '16_stretching',
 '1': '1_nesting',
 '10': '10_scaling_door',
 '6': '6_drinking',
 '17': '17_twitching'}

In [53]:
VIDEO_NAMES

['e3v81a8-20200807T1609-1709.mp4',
 'KDR00048_Top-20220502T201307-211308.mp4',
 'e3v819c-20200808T0609-0709.mp4',
 'e3v819c-20210202T164844-174844.mp4',
 'CAF00106-20210607T110838-120839.mp4',
 'CAF00099-20210608T040846-050846.mp4',
 'KDR00048_Corner-20220502T151301-161302.mp4',
 'e3v81a6-20201230T155846-165847.mp4',
 'e3v81a6-20200915T022243-032244.mp4',
 'e3v81a6-20200915T092249-102249.mp4',
 'e3v819c-20200828T011409-021410.mp4',
 'e3v819c-20200808T0609-0709.mp4',
 'e3v817b-20190705T2055-2155.mp4']

### Load in all videos to corresponding behavior videos

In [59]:
#iterates through all dataframes
for key, value in dataframes.items():
    
    #gets corresponding video
    vid_path = os.path.join(vid_dir, key)
    
    #iterates through each behavior
    for i in range(1, 19):
        behavior_df = value.loc[value['Behavior'] == i]

        beh_path = os.path.join(clip_dir, BEHAVIORS[str(i)])

        #gets the time interval + clips
        for j in range(0, len(behavior_df), 2):
            start_time = behavior_df["Time"].iloc[j]
            end_time = behavior_df["Time"].iloc[j+1]

            name = os.path.join(beh_path, f'{key}_{start_time}_to_{end_time}.mp4')

            ffmpeg_extract_subclip(vid_path, start_time, end_time, targetname = name)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

...

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

### Get nonbehavior clips

In [247]:
#Finds intervals between STOP and STARTS, which indicates nonbehaviors
def non_intervals(df):
    df = df.reset_index()
    
    #finds indexes of STOP
    non_idx = df.index[df['Status'] == 'STOP'].tolist()
    
    intervals = []
    
    for stop in nonbehavior_idx:
        start = stop + 2
        
        if start < len(df):
            #interval between STOP and START
            intervals.append(df.iloc[stop:start]['Time'].values)
        else:
            break
    
    return intervals

In [248]:
all_non_intervals = dict.fromkeys(VIDEO_NAMES)

for video in VIDEO_NAMES:
    all_non_intervals[video] = non_intervals(dataframes[video])

In [249]:
all_non_intervals

{'e3v81a8-20200807T1609-1709.mp4': [array([84.933, 85.467]),
  array([95.533, 97.467]),
  array([123.   , 125.867]),
  array([132.   , 132.733]),
  array([136.467, 138.   ]),
  array([148.867, 152.067]),
  array([156.   , 157.067]),
  array([161.667, 193.8  ]),
  array([246.6, 250.2]),
  array([252.267, 253.333]),
  array([256.267, 259.533]),
  array([265.6  , 266.867]),
  array([267.2  , 267.467]),
  array([272.733, 272.733]),
  array([289.933, 291.   ]),
  array([295.467, 300.   ]),
  array([300., 301.]),
  array([303.933, 307.933]),
  array([315.8  , 317.733]),
  array([319.467, 322.333]),
  array([323.   , 329.267]),
  array([365.8  , 379.067]),
  array([420.4  , 422.467]),
  array([427.   , 513.533]),
  array([521.933, 550.8  ]),
  array([657.8, 672.2]),
  array([697.267, 697.4  ]),
  array([698.6  , 698.733]),
  array([699.267, 699.867]),
  array([700.533, 705.733]),
  array([705.733, 706.467]),
  array([779.467, 784.   ]),
  array([784.   , 789.933]),
  array([790.4  , 793.267])

In [254]:
#iterates through all dataframes
for key, value in all_non_intervals.items():
    
    #gets corresponding video
    vid_path = os.path.join(vid_dir, key)

    #gets the time interval + clips
    for interval in value:
        start_time = interval[0]
        end_time = interval[1]
        
        beh_path = os.path.join(clip_dir, '0_nonbehavior')

        name = os.path.join(beh_path, f'{key}_{start_time}_to_{end_time}.mp4')

        ffmpeg_extract_subclip(vid_path, start_time, end_time, targetname = name)

84.933
85.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
95.533
97.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
123.0
125.867
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
132.0
132.733
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
136.467
138.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
148.867
152.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
156.0
157.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
161.667
193.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
246.6
250.2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
252.267
253.333
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
256.267
259.533
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
265.6
266.867
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


Moviepy - Command successful
1447.533
1447.733
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1449.467
1462.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1463.933
1464.6
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1484.4
1540.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1650.2
1652.667
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1652.667
1666.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1666.933
1669.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1675.133
1679.933
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1690.667
1690.733
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1692.667
1692.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1710.667
1738.733
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1742.867
1743.267
Movi

Moviepy - Command successful
2760.0
2761.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2771.6
2772.133
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2774.933
2775.2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2777.467
2780.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2781.6
2783.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2788.2
2790.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2791.133
2791.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2797.0
2797.2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2799.4
2800.667
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2801.333
2802.4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2803.8
2804.2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2804.4
2804.467
Moviepy - Running:
>

Moviepy - Command successful
246.133
246.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
247.067
247.667
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
247.8
247.867
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
248.2
249.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
249.8
250.933
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
254.733
254.733
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
257.6
257.933
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
258.867
261.667
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
262.267
264.733
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
266.467
266.533
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
269.667
270.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
272.6
272.667
Moviepy - Running:
>>> "+ " ".join

Moviepy - Command successful
1111.133
1111.2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1111.933
1112.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1112.6
1115.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1116.2
1116.333
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1116.933
1118.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1119.4
1119.6
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1120.0
1120.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1120.8
1121.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1123.8
1124.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1125.667
1125.733
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1128.333
1128.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1132.8
1133.0
Moviepy - Running:

...

Moviepy - Command successful
2384.6
2393.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2404.667
2410.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2414.133
2417.4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2425.667
2429.6
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2431.533
2431.933
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2433.0
2434.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2441.0
2441.2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2443.133
2449.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2454.733
2467.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2473.467
2485.6
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2491.0
2493.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2520.0
2539.333
Moviepy - Runnin

Moviepy - Command successful
591.467
591.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
592.4
592.533
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
593.733
593.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
594.2
594.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
594.533
599.933
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
600.467
600.533
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
697.8
750.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
758.467
765.2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
767.333
780.4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
782.0
790.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
790.933
791.133
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
792.667
792.733
Moviepy - Running:
>>> "+ " ".join(cmd

Moviepy - Command successful
1651.8
1652.4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1653.8
1654.533
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1656.733
1658.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1659.733
1660.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1661.2
1662.933
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1664.4
1665.0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1665.6
1666.533
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1668.933
1669.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1674.4
1675.333
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1682.2
1683.533
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1687.533
1688.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1689.333
1690.267
Moviepy - Runnin

Moviepy - Command successful
293.667
298.467
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
323.6
340.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
343.067
346.133
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
348.267
348.4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
354.333
354.933
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
358.6
361.867
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
366.333
372.267
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
375.267
377.067
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
578.2
580.667
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
691.533
695.8
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
710.067
710.333
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
711.933
712.2
Moviepy - Running:
>>> "+ " ".join

Moviepy - Command successful


### Procedure to create test videos

In [270]:
#file that stores test videos
test_dir = os.path.join(cwd, 'behavior_labeling')

In [271]:
#create answer key file in every test folder
for directory in os.listdir(test_dir):
    print(directory)
    if directory.startswith("."):
        continue
    else:
        parent = os.path.join(test_dir, directory)
        f = open(os.path.join(parent, 'answer_key.txt'), 'w')
        f.close()

.DS_Store
test_1
test_3
test_2


In [272]:
#get random videos to use for testing
def get_test_vids(test_dir, clip_dir):
    #for each directory, there should be 2 videos of each behavior
    file_split = len(os.listdir(test_dir)) * 2
    
    test_split = {k: None for k in os.listdir(clip_dir) if os.path.isdir(os.path.join(clip_dir, k))}

    #iterate through every behavior directory in parent directory
    for directory in os.listdir(clip_dir):
        parent = os.path.join(clip_dir, directory)

        #if the file is a directory + there are more than 0 files within directory
        if os.path.isdir(parent) and len(os.listdir(parent)) > 0:
            num_vids = len(os.listdir(parent))

            #if there are less than required number, reuse videos
            if num_vids < file_split:
                diff = file_split - num_vids
                all_files = np.array(range(num_vids))
                diff_files = np.random.randint(low = 0, high = num_vids, size = diff)

                rand_files = np.append(all_files, diff_files)

            #otherwise, get appropriate number
            else:
                rand_files = np.random.randint(low = 0, high = num_vids, size = file_split)
            
            test_split[directory] = rand_files
        
        else:
            test_split[directory] = []
    
    return test_split

In [273]:
def create_test_split(test_dir, clip_dir, test_vids):
    dir_names = [x for x in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, x))]
    test_dirs = dir_names * 2
    
    answers = collections.defaultdict(str)
    
    for key, value in test_vids.items():
        if len(value) > 0:
            for i in range(len(test_dirs)):
                idx = value[i]
                parent_clip = os.path.join(clip_dir, key)
                vid_name = os.listdir(parent_clip)[idx]
                
                parent_test = os.path.join(test_dir, test_dirs[i])
                
                src = os.path.join(parent_clip, vid_name)
                dst = os.path.join(parent_test, vid_name)
                
                shutil.copyfile(src, dst)
                
                answers[test_dirs[i]] += f"{vid_name} {key}\n"
                    
    return answers

In [274]:
def write_answers(answers, test_dir):
    for directory in os.listdir(test_dir):
        parent_dir = os.path.join(test_dir, directory)
        
        if os.path.isdir(parent_dir):
            with open(os.path.join(parent_dir, 'answer_key.txt'), 'w') as text_file:
                text_file.write(answers[directory])

### Execution

In [275]:
test_vids = get_test_vids(test_dir, clip_dir)

answers = create_test_split(test_dir, clip_dir, test_vids)

In [276]:
answers

defaultdict(str,
            {'test_1': 'e3v819c-20210202T164844-174844.mp4_659.867_to_680.9.mp4 11_grooming\ne3v81a6-20201230T155846-165847.mp4_825.833_to_827.133.mp4 11_grooming\ne3v819c-20200808T0609-0709.mp4_338.133_to_338.533.mp4 2_playing_w_ball\ne3v819c-20200828T011409-021410.mp4_1638.0_to_1640.2.mp4 2_playing_w_ball\ne3v81a6-20200915T092249-102249.mp4_2093.2_to_2104.267.mp4 7_chewing\nCAF00106-20210607T110838-120839.mp4_1880.467_to_1885.067.mp4 7_chewing\nCAF00099-20210608T040846-050846.mp4_1509.733_to_1509.833.mp4 0_nonbehavior\ne3v817b-20190705T2055-2155.mp4_889.6_to_891.4.mp4 0_nonbehavior\nKDR00048_Top-20220502T201307-211308.mp4_1892.267_to_1898.733.mp4 5_riding_box\nKDR00048_Top-20220502T201307-211308.mp4_3477.733_to_3488.733.mp4 5_riding_box\nCAF00099-20210608T040846-050846.mp4_3263.4_to_3264.033.mp4 9_scaling_porthole\nCAF00099-20210608T040846-050846.mp4_3263.4_to_3264.033.mp4 9_scaling_porthole\ne3v819c-20200808T0609-0709.mp4_1004.733_to_1006.4.mp4 8_porthole_interactio

### Add in nonbehaviors

In [277]:
nonbehavior_dir = os.path.join(clip_dir, '0_nonbehavior')

In [278]:
num_file = 14
for directory in os.listdir(test_dir):
    
    if directory.startswith(".") == False:
        print("entered")
        test_vids = np.random.randint(low = 0, high = len(os.listdir(nonbehavior_dir)), size = num_file)

        for i in range(0, num_file):
            idx = test_vids[i]
            vid_name = os.listdir(nonbehavior_dir)[idx]

            parent_test = os.path.join(test_dir, directory)

            src = os.path.join(nonbehavior_dir, vid_name)
            dst = os.path.join(parent_test, vid_name)

            shutil.copyfile(src, dst)

            answers[directory] += f"{vid_name} nonbehavior\n"

entered
entered
entered


In [279]:
write_answers(answers, test_dir)